In [ ]:
import os
from tqdm import tqdm

In [ ]:
from avapi.carla import CarlaScenesManager


cpath = os.path.join("/data/shared/CARLA/multi-agent-v1/")
# cpath = "../examples/sim_results"
CSM = CarlaScenesManager(cpath)
print(CSM.scenes)
CDM = CSM.get_scene_dataset_by_index(0)
print(f"{len(CDM)} frames")

In [ ]:
from avapi.visualize.snapshot import show_lidar_bev_with_boxes, show_lidar_on_image, show_image_with_boxes

agent = 0
frame_idx = 60
camera_sensor = "camera-1"
# lidar_sensor = "lidar-0"
frame = CDM.get_frames(sensor=camera_sensor, agent=agent)[frame_idx]
img = CDM.get_image(frame=frame, sensor=camera_sensor, agent=agent)
# pc = CDM.get_lidar(frame=frame, sensor=lidar_sensor, agent=agent)
# objs = CDM.get_objects(frame=frame, sensor=lidar_sensor, agent=agent)
# show_lidar_bev_with_boxes(pc=pc, boxes=objs)
# show_lidar_on_image(img=img, pc=pc, boxes=objs)
im_out = show_image_with_boxes(img=img, boxes=[], inline=True, return_image=True)



## Test Different Perception/Tracking Approaches

In [ ]:
from avstack.geometry import Sphere, Circle

from avstack.modules.perception.object3d import Passthrough3DObjectDetector
from avstack.modules.tracking.tracker3d import BasicBoxTracker3D
from avstack.modules.tracking.multisensor import MeasurementBasedMultiTracker

from avstack.datastructs import DataContainer

# init models
agents = [0, 1]
percep = Passthrough3DObjectDetector()
trackers = {agent: BasicBoxTracker3D() for agent in agents}
trackers["central"] = MeasurementBasedMultiTracker(tracker=BasicBoxTracker3D())

# run loop
dets = {}
tracks = {}
imgs_all = {agent: [] for agent in agents}
dets_all = {agent: [] for agent in agents}
tracks_all = {agent: [] for agent in agents}
tracks_all["central"] = []
agent_0_frames = CDM.get_frames(sensor="lidar-0", agent=0)[1:-1]
for frame in tqdm(agent_0_frames):
    # run perception and individual trackers
    found_data = False
    fovs = {}
    platforms = {}
    for agent in agents:
        ###############################################
        # GET DATA
        ###############################################
        lidar_sensor = "lidar-0"
        camera_sensor = "camera-0"
        img = CDM.get_image(frame=frame, sensor=camera_sensor, agent=agent)
        imgs_all[agent].append(img)
        objs = CDM.get_objects(frame=frame, sensor=lidar_sensor, agent=agent)
        calib = CDM.get_calibration(frame=frame, sensor=lidar_sensor, agent=agent)
        fovs[agent] = Sphere(radius=100)
        platforms[agent] = calib.reference

        ###############################################
        # DISTRIBUTED PERCEPTION
        ###############################################
        found_data = True
        dets[agent] = percep(objs)
        dets_all[agent].append(dets[agent])

        ###############################################
        # DISTRIBUTED TRACKING USING DISTRIBUTED PERCEP
        ###############################################
        tracks[agent] = trackers[agent](dets[agent], platform=calib.reference)
        if not isinstance(tracks[agent], DataContainer):
            raise
        tracks_all[agent].append([track.box3d.copy() for track in tracks[agent]])

    ###############################################
    # CENTRALIZED PERCEPTION
    ###############################################
    ######
    # TODO: QINGZHAO TO COMPLETE
    ######
        
    ###############################################
    # CENTRALIZED TRACKING USING DISTRIBUTED PERCEP
    ###############################################
    # run central tracker on all detections
    if found_data:
        tracks["central"] = trackers["central"](
            detections=dets,
            fovs=fovs,
            platforms=platforms,
        )
        tracks_all["central"].append(tracks["central"])




#### Verify that the union of the two is similar to the central

In [ ]:
from avapi.visualize.snapshot import show_lidar_bev_with_boxes, show_boxes_bev

from avstack.geometry import GlobalOrigin3D

tracks_all = tracks[0]+tracks[1]
tracks_all = [track.change_reference(GlobalOrigin3D, inplace=False) for track in tracks_all]
show_boxes_bev(boxes=tracks_all)
show_boxes_bev(boxes=tracks["central"])

In [ ]:
extent = [[-70, 70], [-70, 70], [-100, 100]]
show_lidar_bev_with_boxes(pc=pc, boxes=[], extent=extent)
show_lidar_bev_with_boxes(pc=pc, boxes=tracks[0], extent=extent)
show_lidar_bev_with_boxes(pc=pc, boxes=tracks_all, extent=extent)
show_lidar_bev_with_boxes(pc=pc, boxes=tracks["central"], extent=extent)

#### Make a video of the results

In [ ]:
from avapi.visualize.movie import make_movie


for agent in agents:

    ###############################################
    # CAMERA-BASED VISUALIZATION
    ###############################################

    # perception movie
    make_movie(
        raw_imgs=imgs_all[agent],
        boxes=dets_all[agent],
        name=f"agent-{agent}-perception-img",
        save=True,
        show_in_notebook=False,
        projection="img",
    )

    # tracking movie
    make_movie(
        raw_imgs=imgs_all[agent],
        boxes=tracks_all[agent],
        name=f"agent-{agent}-tracking-img",
        save=True,
        show_in_notebook=False,
        projection="img",
    )

    ###############################################
    # BEV-BASED VISUALIZATION
    ###############################################

    # perception movie
    make_movie(
        raw_imgs=imgs_all[agent],
        boxes=dets_all[agent],
        name=f"agent-{agent}-perception-bev",
        save=True,
        show_in_notebook=False,
        projection="bev",
    )

    # tracking movie
    make_movie(
        raw_imgs=imgs_all[agent],
        boxes=tracks_all[agent],
        name=f"agent-{agent}-tracking-bev",
        save=True,
        show_in_notebook=False,
        projection="bev",
    )

